In [1]:
import pandas as pd

In [2]:
main_data=pd.read_csv("GEN-sarc-notsarc.csv")
data=main_data.copy()
data.drop(columns=['id'],axis=1,inplace=True)
classes = {"notsarc" : 0,"sarc" : 1}
data["class"] = data["class"].map(classes)
data

,class,text
0,0,"If that's true, then Freedom of Speech is doom..."
1,0,Neener neener - is it time to go in from the p...
2,0,"Just like the plastic gun fear, the armour pie..."
3,0,So geology is a religion because we weren't he...
4,0,Well done Monty. Mark that up as your first ev...
...,...,...
6515,1,depends on when the baby bird died. run alon...
6516,1,"ok, sheesh, to clarify, women who arent aborti..."
6517,1,so.. eh?? hows this sound? will it fly w...
6518,1,"I think we should put to a vote, the right of ..."


In [3]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


from transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from transformers import AdamW

import tensorflow as tf
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

UsageError: Line magic function `%` not found.


In [4]:
sentences = data.text.values
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = data['class'].values

In [5]:
sentences[0:7]

["If that's true, then Freedom of Speech is doomed. Harassment is subjective. Now I can claim that a book I don't like is harassing me, and it will be banned. [SEP] [CLS]",
 'Neener neener - is it time to go in from the playground yet? [SEP] [CLS]',
 'Just like the plastic gun fear, the armour piercing bullet fear was built upon misinformation and myth. [SEP] [CLS]',
 "So geology is a religion because we weren't here to see rock X formed? [SEP] [CLS]",
 'Well done Monty. Mark that up as your first ever 100% honest and accurate post here. [SEP] [CLS]',
 'But the whole process was done in the courts using the process set up by the constitution. Where the laws are not clear on an issue, such as in Massachusetts, it is the courts job to decide. The courts are a check on the injustice towards minorities by the majority. [SEP] [CLS]',
 'so we would penalize financially those who want to have kids? [SEP] [CLS]']

In [6]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Tokenize the first sentence:
['▁if', '▁that', "'", 's', '▁true', ',', '▁then', '▁freedom', '▁of', '▁speech', '▁is', '▁doomed', '.', '▁harassment', '▁is', '▁subjective', '.', '▁now', '▁', 'i', '▁can', '▁claim', '▁that', '▁a', '▁book', '▁', 'i', '▁don', "'", 't', '▁like', '▁is', '▁harass', 'ing', '▁me', ',', '▁and', '▁it', '▁will', '▁be', '▁banned', '.', '▁[', 's', 'ep', ']', '▁[', 'cl', 's', ']']


In [7]:
MAX_LEN = 256

In [8]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [9]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [10]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [11]:

#Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=56, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=56, test_size=0.2)

In [12]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [13]:
# Select a batch size for training. For fine-tuning with XLNet, the authors recommend a batch size of 32, 48, or 128. We will use 32 here to avoid memory issues.
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [14]:

model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased')
model.cuda()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

In [15]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Quadro M4000'

In [17]:
def flat_accuracy(logits,labels):
    return accuracy_score(np.argmax(tf.nn.softmax(logits),axis=1),labels)

In [18]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):


    # Training

    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]
        train_loss_set.append(loss.item())    
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()


        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            logits = output[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))


Epoch:  33%|███▎      | 1/3 [05:41<11:23, 341.63s/it]

Train loss: 0.5653665195579177
Validation Accuracy: 0.7830284552845528


Epoch:  67%|██████▋   | 2/3 [11:22<05:41, 341.42s/it]

Train loss: 0.44582029866294626
Validation Accuracy: 0.7807418699186991


Epoch: 100%|██████████| 3/3 [17:03<00:00, 341.31s/it]

Train loss: 0.3486481459319957
Validation Accuracy: 0.7822662601626016


In [20]:
import os
torch.save(model.state_dict(),os.getcwd()+'/XLNet2.ckpt')

In [ ]:
#load the model
#model = torch.load(PATH)
#model.eval()

In [21]:
# torch.save(model.state_dict(), directory_path+'/model_without_language_model.ckpt')
# Test the model
y_actual=[]
y_pred=[]
with torch.no_grad():
    correct = 0
    total = 0
    for i, batch in enumerate(validation_dataloader):
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        
        prediction = torch.argmax(outputs[0],dim=1)
        y_pred.extend(prediction.cpu().numpy().tolist())
        total += b_labels.size(0)
        y_actual.extend(b_labels.cpu().numpy().tolist())
        correct+=(prediction==b_labels).sum().item()

In [22]:
from sklearn.metrics import classification_report
labels = [0, 1]
print(classification_report(y_actual,y_pred,labels=labels))

              precision    recall  f1-score   support

           0       0.74      0.86      0.79       635
           1       0.84      0.71      0.77       669

    accuracy                           0.78      1304
   macro avg       0.79      0.78      0.78      1304
weighted avg       0.79      0.78      0.78      1304



In [37]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_actual,y_pred)

array([[548,  87],
       [197, 472]])

In [23]:
print('Test Accuracy of the model on vla data is: {} %'.format(100 * correct / total))

Test Accuracy of the model on vla data is: 78.22085889570552 %


In [31]:
test_data=pd.read_csv("HYP-sarc-notsarc.csv")
test_data.drop(columns=['id'],axis=1,inplace=True)
classes = {"notsarc" : 0,"sarc" : 1}
test_data["class"] = test_data["class"].map(classes)
tsentences = test_data.text.values
tsentences = [sentence + " [SEP] [CLS]" for sentence in tsentences]
test_labels = test_data['class'].values
ttokenized_texts = [tokenizer.tokenize(sent) for sent in tsentences]
test_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in ttokenized_texts]
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [32]:
test_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in test_input_ids:
    seq_mask = [float(i>0) for i in seq]
    test_masks.append(seq_mask)

In [33]:
    
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_masks)

test_dataset = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size)

In [34]:
ty_actual=[]
ty_pred=[]
with torch.no_grad():
    correct = 0
    total = 0
    for i, batch in enumerate(test_dataloader):
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        
        prediction = torch.argmax(outputs[0],dim=1)
        ty_pred.extend(prediction.cpu().numpy().tolist())
        total += b_labels.size(0)
        ty_actual.extend(b_labels.cpu().numpy().tolist())
        correct+=(prediction==b_labels).sum().item()

In [35]:
print(classification_report(ty_actual,ty_pred,labels=labels))

              precision    recall  f1-score   support

           0       0.72      0.61      0.66       582
           1       0.66      0.76      0.71       582

    accuracy                           0.69      1164
   macro avg       0.69      0.69      0.69      1164
weighted avg       0.69      0.69      0.69      1164



In [36]:
test_data=pd.read_csv("RQ-sarc-notsarc.csv")
test_data.drop(columns=['id'],axis=1,inplace=True)
classes = {"notsarc" : 0,"sarc" : 1}
test_data["class"] = test_data["class"].map(classes)
tsentences = test_data.text.values
tsentences = [sentence + " [SEP] [CLS]" for sentence in tsentences]
test_labels = test_data['class'].values
ttokenized_texts = [tokenizer.tokenize(sent) for sent in tsentences]
test_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in ttokenized_texts]
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
test_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in test_input_ids:
    seq_mask = [float(i>0) for i in seq]
    test_masks.append(seq_mask)
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_masks)

test_dataset = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size)

ty_actual=[]
ty_pred=[]
with torch.no_grad():
    correct = 0
    total = 0
    for i, batch in enumerate(test_dataloader):
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        
        prediction = torch.argmax(outputs[0],dim=1)
        ty_pred.extend(prediction.cpu().numpy().tolist())
        total += b_labels.size(0)
        ty_actual.extend(b_labels.cpu().numpy().tolist())
        correct+=(prediction==b_labels).sum().item()
print(classification_report(ty_actual,ty_pred,labels=labels))

              precision    recall  f1-score   support

           0       0.75      0.88      0.81       851
           1       0.85      0.70      0.77       851

    accuracy                           0.79      1702
   macro avg       0.80      0.79      0.79      1702
weighted avg       0.80      0.79      0.79      1702



In [38]:
confusion_matrix(ty_actual,ty_pred)

array([[749, 102],
       [254, 597]])